# *SOURCE*
https://www.youtube.com/watch?v=doDUihpj6ro&t=7339s
https://github.com/nicknochnack/ActionDetectionforSignLanguage

In [4]:
! pip install PyQt5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 75.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 MB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 361.8/361.8 KB 38.7 MB/s eta 0:00:00


In [5]:
!pip install tensorflow==2.8.1 tensorflow-gpu==2.8.1 opencv-python mediapipe sklearn 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached tensorflow-2.8.1-cp38-cp38-manylinux2010_x86_64.whl (498.0 MB)
  Using cached tensorflow_gpu-2.8.1-cp38-cp38-manylinux2010_x86_64.whl (498.0 MB)
  Using cached mediapipe-0.9.1.0-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (33.0 MB)
  Using cached sklearn-0.0.post1.tar.gz (3.6 kB)
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 KB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 78.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.3/462.3 KB 49.5 MB/s eta 0:00:00
  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2344 sha256=c2301a07fc594eaca6dc849b66ffc027a96806b17bf1c49a73a8a6a73080d839
  Stored in directory: /root/.cache/pip/wheels/14/25/f7/1cc0956978ae479e75140

In [6]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [8]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('/content/Tutorial_LSTM_MP/Face_data') 

# Actions that we try to detect
actions = np.array(['HadapKanan', 'HadapKiri', 'HadapDepan'])

# Thirty videos worth of data
no_sequences = 30

# Videos are going to be 30 frames in length
sequence_length = 30

In [9]:
! git clone https://github.com/nickbp760/Tutorial_LSTM_MP.git

Cloning into 'Tutorial_LSTM_MP'...
remote: Enumerating objects: 8532, done.
remote: Counting objects: 100% (2795/2795), done.
remote: Compressing objects: 100% (2794/2794), done.
remote: Total 8532 (delta 1), reused 2793 (delta 1), pack-reused 5737
Receiving objects: 100% (8532/8532), 70.43 MiB | 16.83 MiB/s, done.
Resolving deltas: 100% (101/101), done.


### **Preprocess Data and Create Labels and Features**

The label map containts the action label

In [10]:
label_map = {label:num for num, label in enumerate(actions)}

In [11]:
label_map

{'HadapKanan': 0, 'HadapKiri': 1, 'HadapDepan': 2}

load the the data that have been prepared before

In [12]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

1404 is an amount of features in one frame, 30 is an amount frames for one video action, 90 is an amount of 
video actions

In [14]:
468*3

1404

In [13]:
np.array(sequences).shape

(90, 30, 1404)

DEFINE X (Input) and y (Output)

In [15]:
X = np.array(sequences)
print(X.shape)
y = to_categorical(labels).astype(int)
print(y.shape)

(90, 30, 1404)
(90, 3)


In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

### **Build and Train LSTM Neural Network**

In [17]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [18]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1404)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [19]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [20]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])

Epoch 1/2000
3/3 [==============================] - 8s 327ms/step - loss: 256.3913 - categorical_accuracy: 0.4118
Epoch 2/2000
3/3 [==============================] - 1s 276ms/step - loss: 188.0561 - categorical_accuracy: 0.3647
Epoch 3/2000
3/3 [==============================] - 1s 178ms/step - loss: 480.1250 - categorical_accuracy: 0.3882
Epoch 4/2000
3/3 [==============================] - 0s 158ms/step - loss: 518.5331 - categorical_accuracy: 0.4000
Epoch 5/2000
3/3 [==============================] - 0s 160ms/step - loss: 486.7206 - categorical_accuracy: 0.3294
Epoch 6/2000
3/3 [==============================] - 0s 164ms/step - loss: 656.1856 - categorical_accuracy: 0.3647
Epoch 7/2000
3/3 [==============================] - 0s 168ms/step - loss: 2819.1003 - categorical_accuracy: 0.3176
Epoch 8/2000
3/3 [==============================] - 0s 160ms/step - loss: 2102.4097 - categorical_accuracy: 0.3176
Epoch 9/2000
3/3 [==============================] - 0s 149ms/step - loss: 1897.9236 - 

In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 64)            376064    
                                                                 
 lstm_1 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 3)                 99        
                                                                 
Total params: 530,627
Trainable params: 530,627
Non-trai

In [22]:
model.save('/content/Tutorial_LSTM_MP/Weight_model/actionFace.h5')

### **Make One Predictions**

In [23]:
res = model.predict(X_test)

In [24]:

actions[np.argmax(res[4])]

'HadapKiri'

In [25]:
actions[np.argmax(y_test[4])]

'HadapKiri'

### **ReLoad Model**

In [26]:
del model

In [27]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1404)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [28]:
model.load_weights('/content/Tutorial_LSTM_MP/Weight_model/actionFace.h5')

### **Evaluation using Confusion Matrix and Accuracy**

In [29]:
y_test_predict = model.predict(X_test)

In [30]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(y_test_predict, axis=1).tolist()

In [31]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[3, 0],
        [0, 2]],

       [[4, 0],
        [0, 1]],

       [[3, 0],
        [0, 2]]])

In [32]:
accuracy_score(ytrue, yhat)

1.0